In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# 1.x Save Video

In [4]:
cap = cv2.VideoCapture(0)

#비디오 속성 확인
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS) #초당 프레임수

#비디오 레코드 저장
videoWriter= cv2.VideoWriter('train5.avi',cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()
    
    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
        
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF ==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

# 2. Capture Landmarks & Export to CSV

In [2]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [38]:
landmarks=['class']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [40]:
# 데이터 저장할 csv 파일 형성
with open('coords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [3]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        
        if action=='good':
            keypoints = np.insert(keypoints,0,0)

        if action=='bad':
            keypoints = np.insert(keypoints,0,1)
            
        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        pass

In [7]:
# 영상을 통해 데이터셋 구축
cap =cv2.VideoCapture('train5.avi') #setting video capture device(number은 웹캠을 대표하는 숫자)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret,frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        #키보드로 입력한 키와 동일한 아스키코드값
        k = cv2.waitKey(1)
        
        if k == 103: # good의 g
            print('good')
            export_landmark(results,'good')
            
        if k == 98: # bad의 b
            print('bad')
            export_landmark(results,'bad')
            

        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

good
good
good
good
good
bad
bad
bad
bad
bad
bad
bad
bad
bad
bad
bad


# 3. Train Custom Model Using Scikit Learn

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('coords.csv')

In [7]:
df[df['class']==1] #거북목 자세

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,1.0,0.574625,0.645155,-2.716616,0.999971,0.613432,0.541128,-2.666490,0.999928,0.639249,...,0.935618,0.000005,0.643913,3.477567,0.135438,0.000025,0.433251,3.463732,-0.039228,0.000023
5,1.0,0.537767,0.462969,-1.919308,0.999924,0.579951,0.358659,-1.850385,0.999786,0.604060,...,1.004654,0.000023,0.655511,3.420799,0.291147,0.000088,0.431579,3.422243,-0.007665,0.000073
6,1.0,0.521010,0.483275,-1.729316,0.999927,0.563916,0.382079,-1.659376,0.999813,0.590612,...,0.892840,0.000014,0.669275,3.512212,0.104764,0.000052,0.451516,3.521004,-0.025925,0.000051
8,1.0,0.542656,0.566369,-1.812025,0.999912,0.577504,0.468208,-1.772007,0.999773,0.598465,...,1.054646,0.000030,0.678239,3.385082,0.187723,0.000087,0.482416,3.392722,0.219381,0.000050
9,1.0,0.540199,0.485035,-1.278962,0.999912,0.569675,0.393340,-1.198702,0.999779,0.590393,...,0.819317,0.000012,0.678596,3.359066,0.104130,0.000060,0.477530,3.365616,0.017861,0.000036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1.0,0.575684,0.793310,-3.794668,0.999495,0.651851,0.675077,-3.784873,0.999039,0.688868,...,0.995576,0.001793,0.686751,3.586208,0.782773,0.002240,0.507195,3.640984,0.194790,0.003968
154,1.0,0.577354,0.793228,-3.699437,0.999516,0.653100,0.674620,-3.709641,0.999054,0.690517,...,1.088048,0.001945,0.701244,3.634534,0.826685,0.002444,0.502892,3.663879,0.276381,0.004253
155,1.0,0.578887,0.793087,-3.702616,0.999506,0.654823,0.674468,-3.708305,0.998995,0.692543,...,1.100296,0.002151,0.700076,3.634335,0.879401,0.002656,0.486596,3.667595,0.287578,0.004706
156,1.0,0.581935,0.793034,-3.754220,0.999509,0.656170,0.674340,-3.753478,0.998994,0.694430,...,1.130285,0.002271,0.694823,3.631020,0.904441,0.002774,0.480000,3.673965,0.340561,0.005033


In [17]:
X = df.drop('class',axis=1) #class를 제외한 입력 feature만 남김.
y = df['class']

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)

In [22]:
y_test

24     1.0
23     1.0
117    1.0
88     1.0
135    1.0
87     1.0
89     1.0
143    0.0
154    1.0
157    1.0
138    0.0
53     1.0
139    0.0
142    0.0
8      1.0
65     0.0
42     1.0
74     1.0
133    1.0
41     0.0
13     1.0
129    1.0
31     1.0
33     0.0
36     1.0
61     0.0
72     1.0
59     0.0
20     1.0
37     0.0
28     0.0
60     0.0
93     1.0
7      0.0
103    0.0
4      0.0
62     1.0
122    1.0
29     0.0
140    0.0
0      1.0
86     1.0
116    1.0
127    1.0
147    1.0
115    1.0
5      1.0
16     1.0
Name: class, dtype: float64

## 3.2 Train Machine Learning Classification Model

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [30]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(),LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [31]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [32]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [33]:
fit_models['rc'].predict(X_test)

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.])

## 3.3 Evaluate and Serialize Model

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [37]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,pos_label=1),
         recall_score(y_test.values,yhat,pos_label=1))

lr 0.9791666666666666 1.0 0.967741935483871
rc 0.9791666666666666 1.0 0.967741935483871
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [38]:
yhat = fit_models['rf'].predict(X_test)

In [39]:
yhat[:10]

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [40]:
y_test

24     1.0
23     1.0
117    1.0
88     1.0
135    1.0
87     1.0
89     1.0
143    0.0
154    1.0
157    1.0
138    0.0
53     1.0
139    0.0
142    0.0
8      1.0
65     0.0
42     1.0
74     1.0
133    1.0
41     0.0
13     1.0
129    1.0
31     1.0
33     0.0
36     1.0
61     0.0
72     1.0
59     0.0
20     1.0
37     0.0
28     0.0
60     0.0
93     1.0
7      0.0
103    0.0
4      0.0
62     1.0
122    1.0
29     0.0
140    0.0
0      1.0
86     1.0
116    1.0
127    1.0
147    1.0
115    1.0
5      1.0
16     1.0
Name: class, dtype: float64

In [42]:
with open('Textneck.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4. Make Detections with Model

In [43]:
with open('Textneck.pkl','rb') as f:
    model = pickle.load(f)

In [10]:
landmarks=['class']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [11]:
#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Curl counter variables
counter = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        try:  
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7 :
                current_stage = 'Textneck'
                counter +=1
        
            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'Textneck' if body_language_class == 1 else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Stage data 
            cv2.putText(image,'Count',(180,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    
            cv2.putText(image,str(counter),(175,40),
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.